In [1]:
import tensorflow as tf
import numpy as np
import scipy
import time
from tensorflow.keras.preprocessing.image import save_img

# Load the dataset and preprocess the images
def load_dataset(image_size=256, batch_size=16, dataset_path='C:/Users/ASUS/Downloads/faces'):
    def preprocess_image(image):
        image = tf.cast(image, tf.float32)
        image = (image / 127.5) - 1
        return image

    def load_image(image_path):
        image = tf.io.read_file(image_path)
        image = tf.image.decode_jpeg(image, channels=3)
        image = tf.image.resize(image, [image_size, image_size])
        image = preprocess_image(image)
        return image

    def random_crop(image):
        cropped_image = tf.image.random_crop(image, size=[image_size, image_size, 3])
        return cropped_image

  # Load the dataset and apply random cropping
    dataset = tf.data.Dataset.list_files(dataset_path + '/*.png')
    dataset = dataset.map(load_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    dataset = dataset.map(random_crop, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

    return dataset

def generator_model(input_shape):
    inputs = tf.keras.Input(shape=input_shape)
    x = tf.keras.layers.Dense(7*7*256, use_bias=False)(inputs)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.LeakyReLU()(x)
    x = tf.keras.layers.Reshape((7, 7, 256))(x)

    x = tf.keras.layers.Conv2DTranspose(128, (5, 5), strides=(1, 1), padding='same', use_bias=False)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.LeakyReLU()(x)

    x = tf.keras.layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.LeakyReLU()(x)

    x = tf.keras.layers.Conv2DTranspose(3, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh')(x)

    return tf.keras.Model(inputs=inputs, outputs=x)


# # Define the generator and discriminator models
# def generator_model(input_shape, n_residual_blocks=16):
#     inputs = tf.keras.Input(shape=input_shape)

#     # First block: input to residual blocks
#     x = tf.keras.layers.Dense(512)(inputs)
#     x = tf.keras.layers.LeakyReLU(alpha=0.2)(x)
#     x = tf.keras.layers.Reshape((1, 1, 512))(x)

#   # Residual blocks
#     for i in range(n_residual_blocks):
#         x = tf.keras.layers.Conv2D(512, 3, padding='same', use_bias=False)(x)
#         x = tf.keras.layers.BatchNormalization()(x)
#         x = tf.keras.layers.LeakyReLU(alpha=0.2)(x)
#         x = tf.keras.layers.Conv2D(512, 3, padding='same', use_bias=False)(x)
#         x = tf.keras.layers.BatchNormalization()(x)
#         x = tf.keras.layers.LeakyReLU(alpha=0.2)(x)
#         x = tf.keras.layers.Conv2D(256, 3, padding='same', use_bias=False)(x)
#         x = tf.keras.layers.BatchNormalization()(x)
#         x = tf.keras.layers.LeakyReLU(alpha=0.2)(x)

#         # Output
#         x = tf.keras.layers.Conv2D(3, 1, activation='tanh')(x)

#         return tf.keras.Model(inputs=inputs, outputs=x)
    

def discriminator_model(input_shape):
        inputs = tf.keras.Input(shape=input_shape)

        x = tf.keras.layers.Conv2D(64, 3, strides=2, padding='same')(inputs)
        x = tf.keras.layers.LeakyReLU(alpha=0.2)(x)

        x = tf.keras.layers.Conv2D(128, 3, strides=2, padding='same')(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.LeakyReLU(alpha=0.2)(x)

        x = tf.keras.layers.Conv2D(256, 3, strides=2, padding='same')(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.LeakyReLU(alpha=0.2)(x)

        x = tf.keras.layers.Conv2D(512, 3, strides=2, padding='same')(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.LeakyReLU(alpha=0.2)(x)

        # Output
        x = tf.keras.layers.Flatten()(x)
        x = tf.keras.layers.Dense(1, activation='sigmoid')(x)

        return tf.keras.Model(inputs=inputs, outputs=x)

# Define the training loop
def train(dataset, latent_dim=128, epochs=50, sample_interval=50):
    # Create the generator and discriminator models
    generator = generator_model((latent_dim,))
    discriminator = discriminator_model((256, 256, 3))

    # Optimizers
    optimizer_g = tf.keras.optimizers.Adam(0.0002, 0.5)
    optimizer_d = tf.keras.optimizers.Adam(0.0002, 0.5)

    # Loss functions
    loss_fn_g = tf.keras.losses.BinaryCrossentropy(from_logits=True)
    loss_fn_d = tf.keras.losses.BinaryCrossentropy(from_logits=True)

    for epoch in range(epochs):
        start_time = time.time()

        for batch in dataset:
            # Generate fake images
            noise = tf.random.normal((batch.shape[0], latent_dim))
            fake_images = generator(noise, training=True)
            fake_images = tf.image.resize(fake_images, [256, 256])

            # Concatenate real and fake images
            real_and_fake = tf.concat([batch, fake_images], axis=0)

            # Labels for real and fake images
            labels_real = tf.ones((batch.shape[0], 1))
            labels_fake = tf.zeros((batch.shape[0], 1))
            labels = tf.concat([labels_real, labels_fake], axis=0)

            # Train the discriminator
            with tf.GradientTape() as tape:
                logits = discriminator(real_and_fake, training=True)
                loss_d = loss_fn_d(labels, logits)
            grads = tape.gradient(loss_d, discriminator.trainable_variables)
            optimizer_d.apply_gradients(zip(grads, discriminator.trainable_variables))

            # Train the generator
            noise = tf.random.normal((batch.shape[0], latent_dim))
            with tf.GradientTape() as tape:
                fake_images = generator(noise, training=True)
                fake_images = tf.image.resize(fake_images, [256, 256])
                logits_fake = discriminator(fake_images, training=True)
                loss_g = loss_fn_g(tf.ones_like(logits_fake), logits_fake)
            grads = tape.gradient(loss_g, generator.trainable_variables)
            optimizer_g.apply_gradients(zip(grads, generator.trainable_variables))

        # Print the progress
        print("Time for epoch {} is {} sec".format(epoch + 1, time.time() - start_time))

        # Save generated images
        if (epoch + 1) % sample_interval == 0:
            noise = tf.random.normal((16, latent_dim))
            generated_images = generator(noise, training=False)
            for i in range(generated_images.shape[0]):
                save_img('C:/Users/ASUS/Downloads/generated_' + str(i) + '.png', generated_images[i])

In [ ]:
# train the model
train(dataset)

Time for epoch 1 is 10581.954968214035 sec
Time for epoch 2 is 9581.514153718948 sec


In [2]:
# Load the dataset 
dataset = load_dataset()
dataset

<PrefetchDataset element_spec=TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None)>

In [ ]:
# train the model
train(dataset)

Time for epoch 1 is 9208.322114229202 sec
Time for epoch 2 is 9290.493214845657 sec
Time for epoch 3 is 9025.061330080032 sec
Time for epoch 4 is 8755.84211730957 sec
Time for epoch 5 is 8854.225080490112 sec
Time for epoch 6 is 8779.539382696152 sec
Time for epoch 7 is 8773.808583974838 sec
Time for epoch 8 is 8787.269788742065 sec
Time for epoch 9 is 8833.937662601471 sec
Time for epoch 10 is 40082.33596634865 sec


In [10]:
# train the model
train(dataset)

Time for epoch 1 is 26.325513124465942 sec
Time for epoch 2 is 27.825599193572998 sec
Time for epoch 3 is 28.306364059448242 sec
Time for epoch 4 is 28.61549186706543 sec
Time for epoch 5 is 29.49031710624695 sec
Time for epoch 6 is 29.780969381332397 sec
Time for epoch 7 is 30.82942795753479 sec
Time for epoch 8 is 29.02382254600525 sec
Time for epoch 9 is 29.379705667495728 sec
Time for epoch 10 is 29.101566076278687 sec
Time for epoch 11 is 29.461113691329956 sec
Time for epoch 12 is 29.42032551765442 sec
Time for epoch 13 is 30.247260093688965 sec
Time for epoch 14 is 30.669936180114746 sec
Time for epoch 15 is 31.875087022781372 sec
Time for epoch 16 is 30.04021453857422 sec
Time for epoch 17 is 28.960651636123657 sec
Time for epoch 18 is 29.361088514328003 sec
Time for epoch 19 is 29.603143453598022 sec
Time for epoch 20 is 29.148345947265625 sec
Time for epoch 21 is 29.843295335769653 sec
Time for epoch 22 is 30.28518009185791 sec
Time for epoch 23 is 31.34323525428772 sec
Time 